In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

from scipy.stats import spearmanr
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import PredefinedSplit, GridSearchCV

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_lo_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/lo/drd2/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/lo/drd2/test_1.csv', index_col=0)

test

,smiles,value,cluster
0,Brc1ccc(-[n+]2cc[n+](Cc3ccccc3)cc2)c2cc[nH]c12,7.717691,11
1,Brc1cccc(N2CCN(Cc3cnn4ccccc34)CC2)n1,6.748370,26
2,C#CC1=CCC(N(CCC)CCCCn2cc(-c3ccc(-c4ccccc4)cc3)...,6.490481,14
3,C#CCN(CCN1CCN(c2ccccc2)CC1)C1CCc2ccc(O)cc2C1,6.609065,32
4,C1=C(c2ccccc2)CCN(Cc2cnn(-c3ccccc3)c2)C1,7.473269,12
...,...,...,...
262,c1ccc2c(c1)N=C(N1CCNCC1)c1ccccc1S2,7.420216,6
263,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5[nH]4)cc3)CC...,6.568636,35
264,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5o4)cc3)CC2)nc1,6.701147,35
265,c1cnc(N2CCN(Cc3c[nH]c4ncccc34)CC2)nc1,5.931443,12


In [4]:
def spearman_scorer(clf, X, y):
    if len(X) == len(train):
        y_pred = clf.predict(X)
        metrics = get_lo_metrics(train, y_pred)
        return metrics['spearman']
    elif len(X) == len(test):
        y_pred = clf.predict(X)
        metrics = get_lo_metrics(test, y_pred)
        return metrics['spearman']
    else:
        raise ValueError


In [5]:
from scipy.spatial.distance import jaccard


def run_knn_gridsearch_tanimoto(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
        'n_neighbors': [1, 3, 5, 7, 10, 12],
        'weights': ['uniform', 'distance'],
    }
    knn = KNeighborsRegressor(metric=jaccard)

    grid_search = GridSearchCV(knn, params, cv=pds, refit=False, scoring=spearman_scorer, verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    knn = KNeighborsRegressor(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'], metric=jaccard)
    knn.fit(train_fps, train['value'])

    test_preds = knn.predict(test_fps)
    test_metrics = get_lo_metrics(test, test_preds)
    return test_metrics


In [6]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

test_metrics = run_knn_gridsearch_tanimoto(train_morgan_fps, test_morgan_fps)
print(test_metrics)

Fitting 1 folds for each of 12 candidates, totalling 12 fits


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(Spear

[CV 1/1] END ....n_neighbors=1, weights=uniform;, score=0.000 total time=   7.3s


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(Spear

[CV 1/1] END ..n_neighbors=1, weights=distance;, score=-0.041 total time=   7.4s
[CV 1/1] END ....n_neighbors=3, weights=uniform;, score=0.125 total time=   7.5s
[CV 1/1] END ...n_neighbors=3, weights=distance;, score=0.178 total time=   7.5s
[CV 1/1] END ....n_neighbors=5, weights=uniform;, score=0.201 total time=   7.5s
[CV 1/1] END ...n_neighbors=5, weights=distance;, score=0.250 total time=   7.4s
[CV 1/1] END ....n_neighbors=7, weights=uniform;, score=0.191 total time=   7.3s
[CV 1/1] END ...n_neighbors=7, weights=distance;, score=0.196 total time=   7.4s
[CV 1/1] END ...n_neighbors=10, weights=uniform;, score=0.204 total time=   7.4s
[CV 1/1] END ..n_neighbors=10, weights=distance;, score=0.193 total time=   7.4s
[CV 1/1] END ...n_neighbors=12, weights=uniform;, score=0.189 total time=   7.4s
[CV 1/1] END ..n_neighbors=12, weights=distance;, score=0.184 total time=   7.3s
{'n_neighbors': 5, 'weights': 'distance'}
{'r2': -0.33770072764284514, 'spearman': 0.24984819983550363, 'mae'

# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

    knn = KNeighborsRegressor(n_neighbors=5, weights='distance', metric=jaccard)
    knn.fit(train_morgan_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = knn.predict(train_morgan_fps)

    test_result = test.copy()
    test_result['preds'] = knn.predict(test_morgan_fps)

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/lo/drd2/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/lo/drd2/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/lo/drd2/knn_ecfp4/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/lo/drd2/knn_ecfp4/test_{i}.csv')
